In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy import ndimage, misc
import cv2
from skimage.morphology import skeletonize
from fil_finder import FilFinder2D
import astropy.units as u
from sklearn import linear_model
import gc

In [3]:
# import initial image and metrics data
%cd C:\Users\heito\Desktop\Harvard\Sharad Lab\G1\Random_7 
    
img =cv2.imread('random_7_init_crop_masks.tif') #segmented and eroded image
metrics=pd.read_csv('exported_metrics_new.csv')

C:\Users\heito\Desktop\Harvard\Sharad Lab\G1\Random_7


In [19]:
#calculate density, del and del^2 of regions given a window size
window_radius=[100, 250, 400, 500, 600, 750, 1000]
y=metrics.to_numpy()[:,0]
cx=metrics.to_numpy()[:,3]
cy=metrics.to_numpy()[:,4]
# Find Canny edges
edged = cv2.Canny(img, 30, 200)

contours, hierarchy = cv2.findContours(edged, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

number_mat=np.zeros([np.shape(contours)[0],np.size(window_radius)])
density=np.zeros([np.shape(contours)[0],np.size(window_radius)])
grad=np.zeros([np.shape(contours)[0],np.size(window_radius)])
for ii in range(np.shape(contours)[0]):

    repe=np.repeat([[cx[ii],cy[ii]]],np.shape(cx)[0], axis=0)
    dist=np.sqrt((cx[:]-repe[:,0])**2+(cy[:]-repe[:,1])**2)
    for jj in range(np.size(window_radius)):
        number=np.size(np.argwhere(dist<window_radius[jj])) #include self?
        
        number_mat[ii,jj]=number
        density[ii,jj]=number/(np.pi*window_radius[jj]**2)*100**2
        #grad[ii,jj]=1/(np.max(dist[np.argwhere(dist<window_radius[jj])]))
ok=0       
while ok==0:    
    alpha=np.random.rand(1)
    clf = linear_model.Lasso(alpha=alpha)
    clf.fit(density, y)
    if np.size(np.where(clf.coef_!=0))==3:
        ok=1
        
print(clf.coef_)        

C:\Users\heito\anaconda3\lib\site-packages\numpy\core\_asarray.py:102: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return array(a, dtype, copy=False, order=order)


[0.         0.11723626 0.18968253 0.00576337 0.         0.
 0.        ]


In [20]:
alpha

array([0.00021977])

In [100]:
np.where(clf.coef_!=0)

(array([1, 2, 3, 4], dtype=int64),)